In [1]:
from bs4 import BeautifulSoup
import requests
import re
import json
import pandas as pd

In [2]:
url='https://mediabiasfactcheck.com/filtered-search/'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

In [3]:
#content.find(text='getData()')
scripts = soup.find_all('script')
data = scripts[34] #34 is script that includes the JSON file that we want

In [4]:
for elem in soup(text=re.compile("function getData()")):
    data = elem.split("current_json = ", 2)
    data = data[2].split("updateFilters()")
    data = data[0].split(";")
jsondata = data[0] #data[0] contains the json file we are looking for 

In [5]:
result = json.loads(json.dumps(jsondata))
df = pd.DataFrame.from_dict(eval(result), orient='index') #have to do eval(result) because in this case, result isn't the dictionary itself, it's a string representation of a dictionary – eval makes it into dictionary form 
df.head(15)

,b,d,h,L,n,r,u,c
100percentfedup.com,Questionable Sources,100percentfedup.com,https://100percentfedup.com,"97,906",100% Fed up,Low,https://mediabiasfactcheck.com/100-percent-fed...,United States
1010wins.radio.com,Least Biased,1010wins.radio.com,https://1010wins.radio.com,0,1010 WINS AM,High,https://mediabiasfactcheck.com/1010-wins-am/,United States
www.10news.one,Questionable Sources,www.10news.one,https://www.10news.one,"2,011",10News.one,Low,https://mediabiasfactcheck.com/10news-one/,United States
12news.com,Least Biased,12news.com,https://12news.com,0,12 News KPNX,High,https://mediabiasfactcheck.com/12-news-kpnx/,United States
,Left,,https://,0,Vote,Mixed,https://mediabiasfactcheck.com/vote/,
www.whitehouse.gov,Right,www.whitehouse.gov,https://www.whitehouse.gov,"13,742,825",1600 Daily,Mostly Factual,https://mediabiasfactcheck.com/1600-daily/,United States
21stcenturywire.com,Conspiracy-Pseudoscience,21stcenturywire.com,https://21stcenturywire.com,"332,291",21st Century Wire,Mixed,https://mediabiasfactcheck.com/21st-century-wire/,United States
us24news.com,Questionable Sources,us24news.com,https://us24news.com,527,24 News,Low,https://mediabiasfactcheck.com/24-news/,United States
247newsmedia.com,Questionable Sources,247newsmedia.com,https://247newsmedia.com,213,247 News Media,Very Low,https://mediabiasfactcheck.com/247-news-media/,Macedonia
secondamendmentdaily.com,Questionable Sources,secondamendmentdaily.com,https://secondamendmentdaily.com,0,2nd Amendment Daily News,Low,https://mediabiasfactcheck.com/2nd-amendment-d...,United States


In [6]:
df = df.rename(columns={"b": "Bias", "d": "URL", "h": "Full URL", "L": "References", "n": 
                        "Name", "r": "Factualness", "u": "MBFC analysis", "c": "Country"})
df.head()

,Bias,URL,Full URL,References,Name,Factualness,MBFC analysis,Country
100percentfedup.com,Questionable Sources,100percentfedup.com,https://100percentfedup.com,"97,906",100% Fed up,Low,https://mediabiasfactcheck.com/100-percent-fed...,United States
1010wins.radio.com,Least Biased,1010wins.radio.com,https://1010wins.radio.com,0,1010 WINS AM,High,https://mediabiasfactcheck.com/1010-wins-am/,United States
www.10news.one,Questionable Sources,www.10news.one,https://www.10news.one,"2,011",10News.one,Low,https://mediabiasfactcheck.com/10news-one/,United States
12news.com,Least Biased,12news.com,https://12news.com,0,12 News KPNX,High,https://mediabiasfactcheck.com/12-news-kpnx/,United States
,Left,,https://,0,Vote,Mixed,https://mediabiasfactcheck.com/vote/,


In [7]:
print (df['Bias'].unique()) #looking at all the different bias labels

['Questionable Sources' 'Least Biased' 'Left' 'Right'
 'Conspiracy-Pseudoscience' 'Right-Center' 'Left-Center' 'Satire'
 'Pro-Science']


In [33]:
#numerical value for each bias 
bias_dictionary = {"Left": -2, "Left-Center": -1, "Least Biased": 0, "Right-Center": 1, "Right": 2, 
                  "Questionable Sources": 0, "Conspiracy-Pseudoscience": 5, "Satire": -20, "Pro-Science": -5}
df['Bias Rating'] = df['Bias'].map(bias_dictionary)

In [50]:
final_data = pd.read_csv('../data/final_data.csv')
#final_data['Source'].unique()

In [34]:
for name in df['Name'].unique():
    print(name)

100% Fed up
1010 WINS AM
10News.one
12 News KPNX
Vote
1600 Daily
21st Century Wire
24 News
247 News Media
2nd Amendment Daily News
2ndVote
369 News
38 North
3CCorp.net
680 News
71 Republic
7NEWS
9 News (Australia)
911Truth.org
972 Magazine
9News - KUSA
A Sheep No More
A Voice for Men
ABC News
ABC News Australia
ABC11 Eyewitness News
ABC12/WJRT-TV
abcnews-us.com
Abel Danger
Above the Law
Above Top Secret
Abril Uno
ABS-CBN
Acculturated
Accuracy in Academia (AIA)
Accuracy in Media
Ace Flashman
ACHNews
ACLU
ACN Latitudes
Act.TV
Acting Man
Activist Mommy
Activist Post
Ad Fontes Media
Addicting Info
Adobo Chronicles
Advocates for Self-Government
ADW News
Aeon
Affinity Magazine
Africa Check
Africa News
African Arguments
Against Crony Capitalism
Age of Autism
Age of Shitlords
Agence France-Presse (AFP)
Agenzia Nazionale Stampa Associata (ANSA)
Agerpres
Ahram Online
Ahval News
Air and Space Mag
Airwars
AjuaNews.com
Akkadian Times
Akron Beacon Journal
Al Arabiya
Al Bawaba
Al Jazeera
Al Majalla
A

Svijmedia.com
SwampDrain.com
Swarajya
SWI - Swissinfo.ch
Syriana Analysis
Tablet Magazine
Taipei Times
Taiwan News
Take Part
Taki's Magazine
Talking Points Memo
Tallahasee Democrat
Tampa Bay Times
Tampa Tribune
Tarbell
TashNews
Task and Purpose
Tasnim News Agency
Tax Foundation
Tax Justice Network
Tax Policy Center
TD Newswire
TdtAlliance.com
Tea Party Inc.
Tea Party News (the Point)
Tea Party Patriots
Tea Party Tribune
TechCrunch
Techdirt
TechStartups
TechXplore
Teddy Stick
Teen Vogue
Telesur
Tell Me Now
Tenth Amendment Center
Texas Monthly
Texas Standard
Texas Tribune
Tharawat Magazine
That's Nonsense
That's Pretty Good Science
The Advertiser
The Advocate
The Age (Australia)
The Algemeiner Journal
The American Association for the Advancement of Science (AAAS)
The American Cause
The American Conservative
The American Independent Institute
The American Interest
The American Mind
The American Mirror
The American Prospect
The Angry Patriot
The Anti-Media
The Asahi Shimbun
The Atlanta Ban

In [49]:
#find all the sources that are named differently in the final data
unique_sources = final_data[final_data['Source'].isin(df['Name'].unique()) == False]['Source'].unique()
unique_sources

array(['ABC', 'AP', 'BBC', 'BuzzFeed', 'CBS', 'CNSNews', 'Counterpunch',
       'Daily Kos', 'Daily Wire', 'EPI', 'Financial Times', 'Fortune',
       'Forward', 'FreeSpeech TV', 'IJR', 'InfoWars', 'Intercept',
       'Marketwatch', 'National Public Radio', 'NewsPunch', 'NewsMax',
       'OZY', 'Patribotics', 'PBS', 'Progressive, The', 'ProPublica',
       'RedState', 'The Atlantic', 'The New Yorker', 'Time', 'UrNews24',
       'Truthout', 'Vice', 'Washington Free Beacon',
       'Washington Examiner', 'Washington Monthly', 'Washington Post',
       'Weather.com', 'World Truth TV', 'Washington Times', 'Wonkette',
       'WorldNetDaily'], dtype=object)

In [69]:
#change those names to match (this is a temporary solution)
'''
df.replace({'Name': {'ABC News': 'ABC', 'Associated Press': 'AP', 'BBC News': 'BBC', 'Buzzfeed': 'BuzzFeed',
                    'CBS News': 'CBS',
                    'CNS News': 'CNSNews',
                    'CounterPunch': "Counterpunch",
                     '': 'Daily Kos'
                                                                        
                   
                    }}).head(40)
'''
df = df.replace({'Name': {'ABC News': unique_sources[0],
    'Associated Press': unique_sources[1], 
    'BBC News': unique_sources[2],
    'Buzzfeed': unique_sources[3],
    'CBS News': unique_sources[4],
    'CNS News': unique_sources[5],
    'CounterPunch': unique_sources[6],
    'The Daily Wire': unique_sources[8],
    'Economic Policy Institute': unique_sources[9],
    'Financal Times': unique_sources[10],
    'Fortune Magazine': unique_sources[11],
    'The Forward': unique_sources[12],
    'FSTv (Freespeech.org)': unique_sources[13],
    'Independent Journal Review': unique_sources[14],
    'Infowars': unique_sources[15],
    'The Intercept': unique_sources[16],
    'MarketWatch': unique_sources[17],
    'NPR': unique_sources[18],
    'News Punch': unique_sources[19],
    'Newsmax': unique_sources[20],
    'Ozy Media': unique_sources[21],
    'Patribotics Blog': unique_sources[22],
    'PBS News Hour': unique_sources[23],
    'The Progressive': unique_sources[24],
    'Propublica': unique_sources[25],
    'RedStateWave': unique_sources[26],
    'Atlantic': unique_sources[27],
    'New Yorker': unique_sources[28],
    'Time Magazine': unique_sources[29],
    'TruthOut': unique_sources[31],
    'Vice News': unique_sources[32],                   
}})

#wtf no washington post? no washington examiner? no washington ANYTHING? need to add suspiciously missing sources into the data

,Bias,URL,Full URL,References,Name,Factualness,MBFC analysis,Country,Bias Rating
100percentfedup.com,Questionable Sources,100percentfedup.com,https://100percentfedup.com,"97,906",100% Fed up,Low,https://mediabiasfactcheck.com/100-percent-fed...,United States,0
1010wins.radio.com,Least Biased,1010wins.radio.com,https://1010wins.radio.com,0,1010 WINS AM,High,https://mediabiasfactcheck.com/1010-wins-am/,United States,0
www.10news.one,Questionable Sources,www.10news.one,https://www.10news.one,"2,011",10News.one,Low,https://mediabiasfactcheck.com/10news-one/,United States,0
12news.com,Least Biased,12news.com,https://12news.com,0,12 News KPNX,High,https://mediabiasfactcheck.com/12-news-kpnx/,United States,0
,Left,,https://,0,Vote,Mixed,https://mediabiasfactcheck.com/vote/,,-2
www.whitehouse.gov,Right,www.whitehouse.gov,https://www.whitehouse.gov,"13,742,825",1600 Daily,Mostly Factual,https://mediabiasfactcheck.com/1600-daily/,United States,2
21stcenturywire.com,Conspiracy-Pseudoscience,21stcenturywire.com,https://21stcenturywire.com,"332,291",21st Century Wire,Mixed,https://mediabiasfactcheck.com/21st-century-wire/,United States,5
us24news.com,Questionable Sources,us24news.com,https://us24news.com,527,24 News,Low,https://mediabiasfactcheck.com/24-news/,United States,0
247newsmedia.com,Questionable Sources,247newsmedia.com,https://247newsmedia.com,213,247 News Media,Very Low,https://mediabiasfactcheck.com/247-news-media/,Macedonia,0
secondamendmentdaily.com,Questionable Sources,secondamendmentdaily.com,https://secondamendmentdaily.com,0,2nd Amendment Daily News,Low,https://mediabiasfactcheck.com/2nd-amendment-d...,United States,0
